# Installs and Imports

## Libraries

In [ ]:
# For data analysis
import pandas as pd
import numpy as np

# for loading datasets
from google.colab import userdata
import huggingface_hub
from huggingface_hub import HfApi, hf_hub_download, snapshot_download

In [ ]:
hf_hub_download(repo_id="Matthewfung/Work_Major_Extraction",
                repo_type="dataset",
                filename='Original Data/sample_job_ads.xlsx',
                token = userdata.get('HuggingfaceRead'),
                local_dir='/content')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


sample_job_ads.xlsx:   0%|          | 0.00/2.84M [00:00<?, ?B/s]

'/content/Original Data/sample_job_ads.xlsx'

## Data Reading

In [ ]:
df = pd.read_excel('/content/Original Data/sample_job_ads.xlsx')
df.head(3)

,公司名称,关联公司名称,工作地点所在区域,所在区域,所在城市,所在省份,岗位,职位描述,职位类别,待遇,招聘人数,工作经验,学历要求,工作性质,发布日期
0,北京东方天淼商贸有限公司,北京东方天淼商贸有限公司,110105,NaN,朝阳区,北京市,会计主管,岗位职责：1、日常财务核算、会计凭证、出纳、税务工作的审核；2、研究制定会计政策和操作指导，...,财务|人力资源|行政 - 财务/审计/税务 - 会计经理/主管,面议,1人,不限,本科/学士及等同学历 - 本科,全职,2016-03-10
1,成都市第一百惠科技有限公司,成都市第一百惠科技有限公司,511700,NaN,达州市,四川省,区域经理（年薪12万）,主要工作内容：1.负责所辖区域市、县、乡镇手机经销商的开发及维护工作，与客户建立良好的客情关...,销售|客服|市场 - 销售管理 - 销售经理,10001-15000元/月,20人,3-5年,大专及同等学历 - 大专,全职,2014-12-02
2,成都市第一百惠科技有限公司,成都市第一百惠科技有限公司,510105,青羊区,成都市,四川省,收款员,岗位职责：1、公司货款催收及收款；2、其他办公室行政事物处理；3、领导交办的其他临时性事务；...,金融 - 信托/担保/拍卖/典当 - 担保业务,6001-8000元/月,1人,不限,不限,全职,2015-03-27


如果学历要求为“不限”，则职位描述中基本不会有关于“意向学历专业”的描述，所以这里将这部分数据排除。

In [ ]:
data = df[['职位描述', '职位类别', '学历要求']]

data = data[data['学历要求'] != '不限'].reset_index(drop=True)
data = data.rename(columns={"职位描述": "info", "职位类别": "position", "学历要求": "education"})

# QA Models Approach

## Model 1

In [ ]:
# from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
# model = AutoModelForQuestionAnswering.from_pretrained('uer/roberta-base-chinese-extractive-qa')
# tokenizer = AutoTokenizer.from_pretrained('uer/roberta-base-chinese-extractive-qa')

In [ ]:
# QA = pipeline('question-answering', model=model, tokenizer=tokenizer)
# QA_input = {'question': "句子中对学科专业的要求有哪些，如果没有明确给出则返回[ ]",
#             'context': data['info'][0]}

# QA(QA_input)

## Model 2

https://huggingface.co/luhua/chinese_pretrain_mrc_roberta_wwm_ext_large

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "chinese_pretrain_mrc_roberta_wwm_ext_large"
tokenizer = AutoTokenizer.from_pretrained(f"luhua/{model_name}")
model = AutoModelForQuestionAnswering.from_pretrained(f"luhua/{model_name}")
qa = pipeline(task='question-answering',model=model,tokenizer=tokenizer)

config.json:   0%|          | 0.00/634 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.30G [00:00<?, ?B/s]

Some weights of the model checkpoint at luhua/chinese_pretrain_mrc_roberta_wwm_ext_large were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
data['info'][1]

'主要工作内容：1.负责所辖区域市、县、乡镇手机经销商的开发及维护工作，与客户建立良好的客情关系，拉动销量。2.根据市场拓展计划，完成销售目标和拓展目标，并用报表反馈日销量和拓展目标以及一线市场信息。3.定期参加总部的培训和工作统筹计划会议，并根据公司规定及时结清账款和货品。4.定期预约客户上门宣传公司产品，处理客户售后问题，解决客户遇到的各种问题，维护良好的客情关系。5、根据公司的统一部署，做好区域内客户的各种销售促进工作。\xa0任职资格：1.大专及以上文化程度，三年以上优秀销售业绩及相关管理经验，喜欢跑市场，事业心极强、工作务实。2.有坚强的意志，敢于面对困难，具有积极向上的工作心态，优秀的处事能力和客户公关能力，超强的亲和力及耐心，喜欢销售类工作，强烈的获得财富欲望。3.服从管理，具有极强的工作责任心和优秀的执行力，具有吃苦耐劳的意志品质，能适应经常出差（每月休息4天）。4.本人持有C1以上驾照会驾驶优先录取。\xa0薪资及福利年薪12万+各项福利+培训+职位晋升\xa0'

In [ ]:
context = data['info'][2]

def get_answer(txt):

  answer = qa(question = "句子中对相关专业的要求有哪些?",
              context = txt)

  return answer['answer']

In [ ]:
data = data.sample(200).reset_index(drop=True)

In [ ]:
data['majors'] = data['info'].apply(get_answer)

In [ ]:
result1 = data.copy()

In [ ]:
data

,info,position,education,majors
0,河南鼎鑫房产策划公司成立于2012年，是以房屋租赁为主的一家房产管理公司。我们承租业主的房屋...,销售|客服|市场 - 销售业务 - 销售代表,大专及同等学历 - 大专,1、有无经验均可
1,3月19日升格为央企 （注册资金37亿）市场：定位北京高端金融市场员工：高素质、高品质、高绩...,财务|人力资源|行政 - 行政/后勤/文秘 - 行政专员/助理,大专及同等学历 - 大专,不限专业、不要求工作经验
2,工作职责：1、 负责北京市内HP服务器、工作站、笔记本、耗材、打印机、复印机等销售推广工作；...,销售|客服|市场 - 销售业务 - 大客户销售代表,NaN,高中以上学历
3,岗位职责：1.参加行业内展会（上海、广州、北京、郑州、成都等）开发客户2.网络销售：操作淘宝...,销售|客服|市场 - 销售业务 - 网络/在线销售,高中/中专/中技及以下 - 中专,1.半年以上销售经验
4,主要职责：负责医院营销文案编辑，并对官网进行日常更新维护工作根据医院项目的更新，及时编写软文...,服务业 - 医院/医疗/护理 - 其他,大专及同等学历 - 大专,计算机、新闻、中文等相关专业
...,...,...,...,...
195,岗位职责：负责统筹与带领小区开发团队进行小区拓展开发，带领团队完成装潢中心小区开发装潢订单任...,销售|客服|市场 - 销售管理 - 渠道/分销经理/主管,大专及同等学历 - 大专,大专或以上学历
196,招聘要求：1、20—38周岁，身高170CM以上，1年以上管理工作经验；2、身体健康，容貌端...,房产|建筑|物业管理 - 物业管理 - 其他,高中/中专/中技及以下 - 高中,1年以上管理工作经验
197,岗位职责：广告、策划等文稿的书写工作&nbsp;任职要求：文字功底强，知识渊博，沟通、理解及...,销售|客服|市场 - 广告/会展 - 广告文案策划,大专及同等学历 - 大专,中文系
198,岗位职责：1.在总经理的领导下，统筹管理思想家商城线下体验店的全面工作；2.依据公司整体战略...,销售|客服|市场 - 销售管理 - 销售总监,本科/学士及等同学历 - 本科,市场营销、工商管理等相关专业


In [ ]:
result2 = data.copy()

In [ ]:
result2

,info,position,education,majors
0,河南鼎鑫房产策划公司成立于2012年，是以房屋租赁为主的一家房产管理公司。我们承租业主的房屋...,销售|客服|市场 - 销售业务 - 销售代表,大专及同等学历 - 大专,1、有无经验均可
1,3月19日升格为央企 （注册资金37亿）市场：定位北京高端金融市场员工：高素质、高品质、高绩...,财务|人力资源|行政 - 行政/后勤/文秘 - 行政专员/助理,大专及同等学历 - 大专,不限专业、不要求工作经验
2,工作职责：1、 负责北京市内HP服务器、工作站、笔记本、耗材、打印机、复印机等销售推广工作；...,销售|客服|市场 - 销售业务 - 大客户销售代表,NaN,高中以上学历
3,岗位职责：1.参加行业内展会（上海、广州、北京、郑州、成都等）开发客户2.网络销售：操作淘宝...,销售|客服|市场 - 销售业务 - 网络/在线销售,高中/中专/中技及以下 - 中专,1.半年以上销售经验
4,主要职责：负责医院营销文案编辑，并对官网进行日常更新维护工作根据医院项目的更新，及时编写软文...,服务业 - 医院/医疗/护理 - 其他,大专及同等学历 - 大专,计算机、新闻、中文等相关专业
...,...,...,...,...
195,岗位职责：负责统筹与带领小区开发团队进行小区拓展开发，带领团队完成装潢中心小区开发装潢订单任...,销售|客服|市场 - 销售管理 - 渠道/分销经理/主管,大专及同等学历 - 大专,大专或以上学历
196,招聘要求：1、20—38周岁，身高170CM以上，1年以上管理工作经验；2、身体健康，容貌端...,房产|建筑|物业管理 - 物业管理 - 其他,高中/中专/中技及以下 - 高中,1年以上管理工作经验
197,岗位职责：广告、策划等文稿的书写工作&nbsp;任职要求：文字功底强，知识渊博，沟通、理解及...,销售|客服|市场 - 广告/会展 - 广告文案策划,大专及同等学历 - 大专,中文系
198,岗位职责：1.在总经理的领导下，统筹管理思想家商城线下体验店的全面工作；2.依据公司整体战略...,销售|客服|市场 - 销售管理 - 销售总监,本科/学士及等同学历 - 本科,市场营销、工商管理等相关专业
